In [14]:
# Initial imports
import os
import requests
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import matplotlib.pyplot as plt
import hvplot.pandas
%matplotlib inline

In [15]:
# Load .env enviroment variables
load_dotenv()

True

In [16]:
# Set Alpaca API key and secret
alpaca_api_key=os.getenv('ALPACA_KEY')
alpaca_secret_key=os.getenv('ALPACA_SECRET_KEY')
# Create the Alpaca API object
alpaca=tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version='v2'
)

In [17]:
# Define Variables for Alpaca
start_time=pd.Timestamp('2023-07-31', tz='America/New_York').isoformat()
end_time=pd.Timestamp('2023-10-31', tz='America/New_York').isoformat()
tickers = ["AAPL"]
timeframe = "15min"
# Get current closing prices for AAPL and SP500
df_tickers=alpaca.get_bars(
    tickers,
    timeframe,
    start=start_time,
    end=end_time
).df
df_tickers

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2023-07-31 08:00:00+00:00,195.7400,195.90,195.3900,362,195.89,10743,195.671952,AAPL
2023-07-31 08:15:00+00:00,195.6900,195.69,195.6100,44,195.61,1126,195.642842,AAPL
2023-07-31 08:30:00+00:00,195.7500,195.79,195.7100,49,195.74,2434,195.754051,AAPL
2023-07-31 08:45:00+00:00,195.8100,195.81,195.6200,133,195.75,5845,195.727415,AAPL
2023-07-31 09:00:00+00:00,195.9100,195.95,195.7500,181,195.78,5860,195.840075,AAPL
...,...,...,...,...,...,...,...,...
2023-10-30 22:45:00+00:00,169.8699,169.90,169.8200,41,169.82,4060,169.863674,AAPL
2023-10-30 23:00:00+00:00,169.8200,169.91,169.8200,112,169.87,5285,169.867104,AAPL
2023-10-30 23:15:00+00:00,169.8500,169.87,169.8500,97,169.85,3450,169.861542,AAPL


In [18]:
# Defining variables
shares = 100

In [19]:
# Reorganize the DataFrame
df_tickers.drop(['high','low','trade_count','volume','vwap','symbol','open'], axis=1, inplace=True)

In [20]:
### Establishing control benchmark
df_tickers['return'] = df_tickers['close'].pct_change() + 1
df_tickers['return'].iat[0] = 1
fund = shares*df_tickers['close'][0]
df_tickers['benchmark'] = fund * df_tickers['return'].cumprod()
df_tickers


,close,return,benchmark
timestamp,,,
2023-07-31 08:00:00+00:00,195.7400,1.000000,19574.00
2023-07-31 08:15:00+00:00,195.6900,0.999745,19569.00
2023-07-31 08:30:00+00:00,195.7500,1.000307,19575.00
2023-07-31 08:45:00+00:00,195.8100,1.000307,19581.00
2023-07-31 09:00:00+00:00,195.9100,1.000511,19591.00
...,...,...,...
2023-10-30 22:45:00+00:00,169.8699,0.999823,16986.99
2023-10-30 23:00:00+00:00,169.8200,0.999706,16982.00
2023-10-30 23:15:00+00:00,169.8500,1.000177,16985.00


In [21]:
### Creating new Dataframe for MACD calculations
df_macd = pd.DataFrame()
df_macd['MACD'] =  df_tickers['close'].ewm(halflife=12).mean() - df_tickers['close'].ewm(halflife=26).mean()
df_macd['Signal'] = df_macd['MACD'].ewm(halflife=9).mean()
df_macd['histogram'] = df_macd['MACD'] - df_macd['Signal']
df_macd

,MACD,Signal,histogram
timestamp,,,
2023-07-31 08:00:00+00:00,0.000000,0.000000,0.000000
2023-07-31 08:15:00+00:00,-0.000389,-0.000202,-0.000187
2023-07-31 08:30:00+00:00,0.000120,-0.000086,0.000206
2023-07-31 08:45:00+00:00,0.001084,0.000241,0.000843
2023-07-31 09:00:00+00:00,0.002933,0.000865,0.002067
...,...,...,...
2023-10-30 22:45:00+00:00,0.449925,0.483726,-0.033801
2023-10-30 23:00:00+00:00,0.437851,0.480326,-0.042475
2023-10-30 23:15:00+00:00,0.427002,0.476373,-0.049371


In [22]:
### Plotting Close price
Close_plot = df_tickers['close'].hvplot.line(    
    x='timestamp', 
    y='close',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='Closing Price AAPL',
    grid=True,
).opts(
    yformatter='%.0f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)
### Plotting MACD LINE & signal line
MACD_plot = df_macd['MACD'].hvplot.line(    
    x='timestamp', 
    y='MACD',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='MACD Line & Signal Line',
    grid=True,
).opts(
    yformatter='%.2f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)
Signal_plot = df_macd['Signal'].hvplot.line(    
    x='timestamp', 
    y='Signal',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='AAPL close', 
    xlabel='timestamp',
    title='MACD Line & Signal Line',
    grid=True,
    legend=True,
).opts(
    yformatter='%.2f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)
### Plotting Histogram
Histogram_plot = df_macd['histogram'].hvplot.bar(    
    x='timestamp', 
    y='histogram',
    rot=90,
    frame_width=1000,
    frame_height=500,
    ylabel='MACD-Signal', 
    xlabel='timestamp',
    title='MACD Line - Signal Line',
    grid=True,
    legend=True,
).opts(
    yformatter='%.2f',
    hover_color="orange",
    gridstyle={'color': 'gray', 'line_width': 1, 'line_style': 'solid'}
)
### look up HVPLOT or convert to utc time in the dataframe.


In [23]:
display(Close_plot)
display(MACD_plot * Signal_plot)
Histogram_plot

:Curve   [timestamp]   (close)

:Overlay
   .Curve.MACD   :Curve   [timestamp]   (MACD)
   .Curve.Signal :Curve   [timestamp]   (Signal)

:Bars   [timestamp]   (histogram)

### I HAVE CALCULATED THE MACD LINE THE SIGNAL LINE
### I WILL BE USING A CROSSOVER STRATEGY TO MAKE MONEY ON IT.
Investopedia
Crossover Strategy
A crossover occurs when the signal and MACD line cross each other. The MACD generates a bullish signal when it moves above its own nine-day EMA and sends a sell signal (bearish) when it moves below its nine-day EMA.

In [24]:
# Creating signals
# Finding long or short trades
df_macd["buy/sell"]=np.where(df_macd["MACD"]>df_macd["Signal"],1,0) 
df_macd["buy/sell"]=df_macd["buy/sell"].diff() 
df_macd['close'] = df_tickers['close']
display(df_macd)



,MACD,Signal,histogram,buy/sell,close
timestamp,,,,,
2023-07-31 08:00:00+00:00,0.000000,0.000000,0.000000,NaN,195.7400
2023-07-31 08:15:00+00:00,-0.000389,-0.000202,-0.000187,0.0,195.6900
2023-07-31 08:30:00+00:00,0.000120,-0.000086,0.000206,1.0,195.7500
2023-07-31 08:45:00+00:00,0.001084,0.000241,0.000843,0.0,195.8100
2023-07-31 09:00:00+00:00,0.002933,0.000865,0.002067,0.0,195.9100
...,...,...,...,...,...
2023-10-30 22:45:00+00:00,0.449925,0.483726,-0.033801,0.0,169.8699
2023-10-30 23:00:00+00:00,0.437851,0.480326,-0.042475,0.0,169.8200
2023-10-30 23:15:00+00:00,0.427002,0.476373,-0.049371,0.0,169.8500


In [25]:
# Extracting the timestamps that indicate bullish signals (Signals where we buy) and that indicate bearish signals (signals where we sell)
Xreturns=df_macd[df_macd["buy/sell"].isin([1,-1])] 
# Dataframe with trade signals and dates
Xreturns['Lcashflow'] = -Xreturns['buy/sell'] * Xreturns['close'] * shares
Xreturns['Scashflow'] = Xreturns['buy/sell'] * Xreturns['close'] * shares
cashflowl = Xreturns["Lcashflow"].sum()
cashflows = Xreturns["Scashflow"].sum()
display(Xreturns)
display(cashflowl)

c:\Users\Micha\anaconda3\envs\project\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\Users\Micha\anaconda3\envs\project\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,MACD,Signal,histogram,buy/sell,close,Lcashflow,Scashflow
timestamp,,,,,,,
2023-07-31 08:30:00+00:00,0.000120,-0.000086,0.000206,1.0,195.7500,-19575.00,19575.00
2023-07-31 09:30:00+00:00,0.000952,0.001168,-0.000216,-1.0,195.7000,19570.00,-19570.00
2023-07-31 10:15:00+00:00,0.002546,0.001143,0.001403,1.0,195.8900,-19589.00,19589.00
2023-07-31 16:00:00+00:00,0.024745,0.028052,-0.003307,-1.0,195.7850,19578.50,-19578.50
2023-07-31 17:45:00+00:00,0.027605,0.025967,0.001638,1.0,196.2399,-19623.99,19623.99
...,...,...,...,...,...,...,...
2023-10-25 08:45:00+00:00,0.016455,0.023610,-0.007155,-1.0,172.5900,17259.00,-17259.00
2023-10-26 12:30:00+00:00,-0.731314,-0.745241,0.013928,1.0,170.4700,-17047.00,17047.00
2023-10-26 15:00:00+00:00,-0.721525,-0.688653,-0.032872,-1.0,167.3800,16738.00,-16738.00


-2239.5100000000384

In [26]:
### Cashflow is being calculated as the sum of all outflows and inflows of cash. The outflows are when we are buying, that inflows are when we are selling.
### The benchmark is being calculated as a result of the initial value of 100 shares defined in the fund variable minus the last value in the df_tickers['benchmark'] column.
print(f"Profit/Loss MACD LONG STRATEGY: {cashflowl}\nProfit/Loss MACD Short Strategy: {cashflows}\nProfit/Loss Benchmark: {df_tickers['benchmark'][-1] - fund}")

Profit/Loss MACD LONG STRATEGY: -2239.5100000000384
Profit/Loss MACD Short Strategy: 2239.5100000000384
Profit/Loss Benchmark: -2603.9999999999454
